In [12]:
import os
from pathlib import Path

path = Path('data/bodyfat_dataset.csv')


In [13]:
import torch, numpy as np, pandas as pd

df = pd.read_csv(path)

df

,title,url,meanPrediction,medianPrediction,bfPredictions,image_1,image_2,image_3,image_4,image_5
0,Leanest ive ever been. Never seen veins like this before.,https://www.reddit.com/gallery/1b6k5jh,8.00,8.0,[8],https://preview.redd.it/leanest-ive-ever-been-never-seen-veins-like-this-before-v0-hibkcr8hfdmc1.jpg?width=640&crop=smart&auto=webp&s=4604d036777deffcf28c2e2f396b19f77ac54c30,https://preview.redd.it/leanest-ive-ever-been-never-seen-veins-like-this-before-v0-cvvyjr8hfdmc1.jpg?width=640&crop=smart&auto=webp&s=bbc6e7207b1aae8893ecb76d1a68adc87fcd7aa2,https://preview.redd.it/leanest-ive-ever-been-never-seen-veins-like-this-before-v0-kev0ir8hfdmc1.jpg?width=640&crop=smart&auto=webp&s=3cf034949f12a0b62655c3dbcd0d77d4418c64fe,https://preview.redd.it/leanest-ive-ever-been-never-seen-veins-like-this-before-v0-yur9ot8hfdmc1.jpg?width=476&format=pjpg&auto=webp&s=5af3f2a2258f511a1606651727635f31f55f4d82,NaN
1,Let me know. 78kg :),https://i.redd.it/4occeq9wdd2c1.jpg,9.80,10.0,"[9, 10, 10, 9, 11]",https://i.redd.it/4occeq9wdd2c1.jpg,NaN,NaN,NaN,NaN
2,What is my bf% I believe it’s around 13-14%,https://i.redd.it/lvfmowq0zhh91.jpg,14.33,13.5,"[13, 19, 14, 14, 13, 13]",https://i.redd.it/lvfmowq0zhh91.jpg,NaN,NaN,NaN,NaN
3,"25F | 4'11"" | 107 lbs",https://www.reddit.com/gallery/1e30z2f,23.33,24.0,"[20, 26, 24]",https://preview.redd.it/25f-411-107-lbs-v0-4lmyvzck8hcd1.jpg?width=640&crop=smart&auto=webp&s=8ad8d28fdda2a0c57664d858e31e13889dfb008d,https://preview.redd.it/25f-411-107-lbs-v0-huczvrhk8hcd1.jpg?width=640&crop=smart&auto=webp&s=2445bde27112796d8e09d8825a1cb03d7bb83108,NaN,NaN,NaN
4,Bodyfat?,https://www.reddit.com/gallery/1ktwao9,17.80,18.0,"[18, 16, 18, 15, 22]",https://preview.redd.it/8ekhv0d4xl2f1.jpg?width=640&crop=smart&auto=webp&s=dfaa3a2d96c714e36a6b2579ad6559b098e145c7,https://preview.redd.it/rwkt50d4xl2f1.jpg?width=640&crop=smart&auto=webp&s=9d4a08637bbd189e9f44f47dfe20948be74f85d4,https://preview.redd.it/vau412d4xl2f1.jpg?width=640&crop=smart&auto=webp&s=610a51fef6107a8c7635afdc1722d01e620bf86b,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
800,[GMBF] 6'1 220LBS Any estimates appreciated!,https://i.redd.it/z543jfquva731.jpg,12.00,12.0,[12],https://i.redd.it/z543jfquva731.jpg,NaN,NaN,NaN,NaN
801,"[GMBF] (M/22/6'2""/195lbs)",https://i.imgur.com/PY44dK5.jpg,10.50,10.5,"[11, 9, 10, 12]",https://i.imgur.com/PY44dK5.jpg,NaN,NaN,NaN,NaN
802,[GMBF] M/26/6'1/191lbs - down from 245lbs,https://i.imgur.com/pWGT7nV.jpg,15.00,15.0,[15],https://i.imgur.com/pWGT7nV.jpg,NaN,NaN,NaN,NaN
803,[GMBF] (M/26/5’10”/153lbs to 168lbs) What would you estimate my bf% to be before and after 4 months of (hopefully lean) bulking?,https://i.redd.it/lc2q2svumrt11.jpg,12.00,12.0,"[10, 14]",https://i.redd.it/lc2q2svumrt11.jpg,NaN,NaN,NaN,NaN


In [6]:
import requests
from tqdm import tqdm

def download_all_images(df, output_dir="raw_images"):
    os.makedirs(output_dir, exist_ok=True)
    image_cols = [f"image_{i}" for i in range(1, 6)]
    
    seen_urls = set()
    image_count = 0

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        for col in image_cols:
            url = row.get(col)
            if isinstance(url, str) and url.startswith("http") and url not in seen_urls:
                try:
                    response = requests.get(url, timeout=10)
                    if response.status_code == 200:
                        file_ext = url.split('.')[-1].split('?')[0]
                        file_name = f"{idx}_{col}.{file_ext}"
                        file_path = os.path.join(output_dir, file_name)
                        with open(file_path, "wb") as f:
                            f.write(response.content)
                        seen_urls.add(url)
                        image_count += 1
                except Exception as e:
                    print(f"Error downloading {url}: {e}")

    print(f"\nDownloaded {image_count} unique images to '{output_dir}/'")

In [15]:
def create_regression_csv(df, output_csv="image_labels.csv", label_col="meanPrediction", image_prefix="image_", output_dir="images"):
    # Ensure column names are stripped of whitespace
    df.columns = df.columns.str.strip()
    
    image_cols = [col for col in df.columns if col.startswith(image_prefix)]
    records = []

    for idx, row in df.iterrows():
        label = row[label_col]
        for col in image_cols:
            url = row.get(col)
            if isinstance(url, str) and url.startswith("http"):
                ext = url.split('.')[-1].split('?')[0].lower()
                ext = ext if ext in ['jpg', 'jpeg', 'png', 'webp'] else 'jpg'
                filename = f"{idx}_{col}.{ext}"
                records.append({"filename": filename, "target": label})
    
    df_out = pd.DataFrame(records)
    df_out.to_csv(output_csv, index=False)
    print(f"Created {output_csv} with {len(df_out)} labeled images")
    return df_out

In [16]:
download_all_images(df)  

df_labels = create_regression_csv(df) 
df_labels.head()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 805/805 [03:21<00:00,  3.99it/s]


Downloaded 1578 unique images to 'raw_images/'
Created image_labels.csv with 1596 labeled images


,filename,target
0,0_image_1.jpg,8.0
1,0_image_2.jpg,8.0
2,0_image_3.jpg,8.0
3,0_image_4.jpg,8.0
4,1_image_1.jpg,9.8


In [7]:
from fastai.vision.all import *

failed = verify_images(get_image_files(Path('raw_images')))
failed.map(Path.unlink)
len(failed)

0

In [8]:
!python3.10 -m pip install -Uqq mediapipe

In [12]:
import mediapipe as mp
from PIL import Image
import cv2


# Paths to your datasets and images
data_dir = Path('./data')
image_labels_csv = data_dir / 'image_labels.csv'
bodyfat_csv = data_dir / 'bodyfat_dataset.csv'
images_dir = Path('raw_images')  # Adjust this path if your images are elsewhere

# Load CSVs
df_labels = pd.read_csv(image_labels_csv)
df_bodyfat = pd.read_csv(bodyfat_csv)

print(f"Labels df shape: {df_labels.shape}")
print(f"Bodyfat df shape: {df_bodyfat.shape}")

# Setup mediapipe pose detector
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)


Labels df shape: (1596, 3)
Bodyfat df shape: (805, 10)


I0000 00:00:1749707333.470032 4530522 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1749707333.523661 4538337 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749707333.532377 4538346 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [13]:
def crop_upper_body(image_path):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Failed to read image {image_path}")
        return None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    if not results.pose_landmarks:
        print(f"No pose detected in {image_path}")
        return None

    try:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]

        h, w, _ = img.shape
        xs = [left_shoulder.x * w, right_shoulder.x * w, left_hip.x * w, right_hip.x * w]
        ys = [left_shoulder.y * h, right_shoulder.y * h, left_hip.y * h, right_hip.y * h]

        xmin, xmax = int(min(xs)), int(max(xs))
        ymin, ymax = int(min(ys)), int(max(ys))

        # Add padding around landmarks
        pad_x = 20
        pad_y = 20
        xmin = max(xmin - pad_x, 0)
        xmax = min(xmax + pad_x, w)
        ymin = max(ymin - pad_y, 0)
        ymax = min(ymax + pad_y, h)

        # Final adjustment: widen the crop a little more (e.g., 10% of width)
        box_width = xmax - xmin
        expand = int(box_width * 0.1)  # Expand 10% on each side
        xmin = max(xmin - expand, 0)
        xmax = min(xmax + expand, w)

        cropped = img[ymin:ymax, xmin:xmax]
        cropped_rgb = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
        return Image.fromarray(cropped_rgb)

    except Exception as e:
        print(f"Error cropping {image_path}: {e}")
        return None


In [14]:
# Create folder for cropped images if it doesn't exist
cropped_dir = Path('images')
cropped_dir.mkdir(exist_ok=True)

cropped_paths = []
failed_images = []

for idx, row in tqdm(df_labels.iterrows(), total=len(df_labels)):
    filename = row['filename']
    img_path = images_dir / filename
    cropped_img = crop_upper_body(img_path)
    if cropped_img is not None:
        save_path = cropped_dir / filename
        cropped_img.save(save_path)
        cropped_paths.append(str(save_path))
    else:
        failed_images.append(filename)
        cropped_paths.append(None)  # To keep alignment with df_labels rows

print(f"Done! Cropped {len([p for p in cropped_paths if p])} images, failed on {len(failed_images)} images.")

# Add cropped image paths to dataframe and save
df_labels['cropped_path'] = cropped_paths
df_labels.to_csv(data_dir / 'image_labels.csv', index=False)


  1%|▍                                                                                                 | 8/1596 [00:00<00:47, 33.49it/s]

No pose detected in raw_images/0_image_1.jpg
No pose detected in raw_images/0_image_2.jpg
No pose detected in raw_images/0_image_3.jpg
No pose detected in raw_images/2_image_1.jpg


  1%|▋                                                                                                | 12/1596 [00:00<00:59, 26.50it/s]

No pose detected in raw_images/4_image_2.jpg
No pose detected in raw_images/4_image_3.jpg
No pose detected in raw_images/5_image_1.png
No pose detected in raw_images/7_image_1.jpg


[ WARN:0@52.537] global loadsave.cpp:268 findDecoder imread_('raw_images/8_image_1.jpg'): can't open/read file: check file path/integrity
  2%|█▍                                                                                               | 24/1596 [00:00<00:40, 38.72it/s]

...

No pose detected in raw_images/773_image_1.jpg
Failed to read image raw_images/773_image_2.jpg
Failed to read image raw_images/773_image_3.jpg
Failed to read image raw_images/773_image_4.jpg
Failed to read image raw_images/776_image_1.jpg
Failed to read image raw_images/776_image_2.jpg
Failed to read image raw_images/776_image_3.jpg
Failed to read image raw_images/777_image_1.png
Failed to read image raw_images/779_image_1.jpg
Failed to read image raw_images/779_image_2.jpg
Failed to read image raw_images/780_image_1.jpg
Failed to read image raw_images/782_image_1.jpg
Failed to read image raw_images/782_image_2.jpg
Failed to read image raw_images/782_image_3.jpg
Failed to read image raw_images/782_image_4.jpg
Failed to read image raw_images/783_image_1.png
Failed to read image raw_images/783_image_2.png
Failed to read image raw_images/783_image_3.png
Failed to read image raw_images/783_image_4.png
Failed to read image raw_images/785_image_3.jpg
Failed to read image raw_images/785_image

[ WARN:0@78.124] global loadsave.cpp:268 findDecoder imread_('raw_images/785_image_3.jpg'): can't open/read file: check file path/integrity
[ WARN:0@78.124] global loadsave.cpp:268 findDecoder imread_('raw_images/785_image_4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@78.177] global loadsave.cpp:268 findDecoder imread_('raw_images/786_image_2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@78.276] global loadsave.cpp:268 findDecoder imread_('raw_images/789_image_2.jpg'): can't open/read file: check file path/integrity
 99%|█████████████████████████████████████████████████████████████████████████████████████████████▋ | 1574/1596 [00:26<00:00, 82.17it/s]

Failed to read image raw_images/786_image_2.jpg
Failed to read image raw_images/789_image_2.jpg
No pose detected in raw_images/791_image_2.jpg


[ WARN:0@78.449] global loadsave.cpp:268 findDecoder imread_('raw_images/793_image_2.jpg'): can't open/read file: check file path/integrity
 99%|██████████████████████████████████████████████████████████████████████████████████████████████▎| 1584/1596 [00:26<00:00, 77.60it/s][ WARN:0@78.449] global loadsave.cpp:268 findDecoder imread_('raw_images/793_image_3.jpg'): can't open/read file: check file path/integrity
[ WARN:0@78.449] global loadsave.cpp:268 findDecoder imread_('raw_images/793_image_4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@78.450] global loadsave.cpp:268 findDecoder imread_('raw_images/794_image_1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@78.450] global loadsave.cpp:268 findDecoder imread_('raw_images/795_image_1.png'): can't open/read file: check file path/integrity
[ WARN:0@78.593] global loadsave.cpp:268 findDecoder imread_('raw_images/801_image_1.jpg'): can't open/read file: check file path/integrity
100%|███████████████████

Failed to read image raw_images/793_image_2.jpg
Failed to read image raw_images/793_image_3.jpg
Failed to read image raw_images/793_image_4.jpg
Failed to read image raw_images/794_image_1.jpg
Failed to read image raw_images/795_image_1.png
No pose detected in raw_images/800_image_1.jpg
Failed to read image raw_images/801_image_1.jpg
Failed to read image raw_images/802_image_1.jpg
Failed to read image raw_images/804_image_1.jpg
Failed to read image raw_images/805_image_1.jpeg
Done! Cropped 892 images, failed on 704 images.
